In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import pandas
import numpy
from matplotlib import pyplot as plt
import seaborn
%matplotlib notebook
import datetime
import general
import discretionary_aua
import vantage_aua
import data_accessing
import combined
import revenue
import costs
import consolidated
import discf
from ipywidgets import widgets
from IPython.display import display
import stats
from IPython.display import clear_output
pandas.set_option('display.float_format', '{:20,.6f}'.format)   #suppressing scientific notation
idx = pandas.IndexSlice

In [3]:
print('Pulling data...')
fund_dic = data_accessing.read_data(data_accessing.fund_data_name,data_accessing.fund_data_types)
nnb_dic = data_accessing.read_data(data_accessing.nnb_data_name,data_accessing.nnb_data_sheet)
discretionary_aua.append_share_class_units(fund_dic) # appending share class units
index_hl_dic = data_accessing.read_data(data_accessing.index_hl_data_name, data_accessing.index_hl_data_sheet)
data_dic = {**fund_dic, **index_hl_dic, **nnb_dic}

input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)

print('Data pulled successfully.')

Pulling data...
Data pulled successfully.


In [4]:
assumption_button = widgets.Button(description='Update assumptions',layout=widgets.Layout(width='20%',height='40px'))
assumption_button.style.button_color = 'lightgreen'
def update_assumption(b):
    global input_dic
    input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)
    print ('Assumptions updated successfully...')

assumption_button.on_click(update_assumption)
display(assumption_button)

Button(description='Update assumptions', layout=Layout(height='40px', width='20%'), style=ButtonStyle(button_color='lightgreen'))

In [5]:
revenue_types = ['Semi-annual revenue','Annual revenue - FY','Annual revenue - CY','Consolidated semi-annual revenue','Consolidated annual revenue - FY','Consolidated annual revenue - CY']
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch',border='solid',width='30%')
items = {}
for w in revenue_types:
    items[w] = widgets.Button(description=w,layout=widgets.Layout(width='100%', height='40px'))
    items[w].style.button_color = 'lightgreen'
left_box = widgets.VBox([list(items.values())[0],list(items.values())[1],list(items.values())[2]],layout=box_layout)
right_box = widgets.VBox([list(items.values())[3],list(items.values())[4],list(items.values())[5]],layout=box_layout)
items['Semi-annual revenue'].on_click(lambda x: display(revenue.semi_revenue(data_dic,input_dic)))
items['Annual revenue - FY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic)))
items['Annual revenue - CY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic,cal_year=True)))
items['Consolidated semi-annual revenue'].on_click(lambda x: display(consolidated.revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - FY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - CY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic,cal_year=True)))
widgets.HBox([left_box,right_box])



HBox(children=(VBox(children=(Button(description='Semi-annual revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='Consolidated semi-annual revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual revenue - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual revenue - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

In [6]:
costs_types = ['Semi-annual costs','Annual costs - FY','Annual costs - CY','Consolidated semi-annual costs','Consolidated annual costs - FY','Consolidated annual costs - CY']
items2 = {}
for w2 in costs_types:
    items2[w2] = widgets.Button(description=w2,layout=widgets.Layout(width='100%', height='40px'))
    items2[w2].style.button_color = 'lightgreen'
left_box2 = widgets.VBox([list(items2.values())[0],list(items2.values())[1],list(items2.values())[2]],layout=box_layout)
right_box2 = widgets.VBox([list(items2.values())[3],list(items2.values())[4],list(items2.values())[5]],layout=box_layout)
items2['Semi-annual costs'].on_click(lambda x: display(costs.semi_costs(input_dic)))
items2['Annual costs - FY'].on_click(lambda x: display(costs.annual_costs(input_dic)))
items2['Annual costs - CY'].on_click(lambda x: display(costs.annual_costs(input_dic,cal_year=True)))
items2['Consolidated semi-annual costs'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - FY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic)))
items2['Consolidated annual costs - CY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic,cal_year=True)))
widgets.HBox([left_box2,right_box2])

HBox(children=(VBox(children=(Button(description='Semi-annual costs', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='Consolidated semi-annual costs', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual costs - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual costs - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

,Staff costs,Marketing and distribution spend,"Depreciation, amortisation & financial costs",Office running costs,Other costs,FSCS levy costs,Total operating costs
financial_year,,,,,,,
2018,"-84,619,920.000000","-13,970,430.000000","-8,908,680.000000","-5,871,630.000000","-30,775,440.000000","-4,500,000.000000","-148,646,100.000000"
2019,"-88,838,920.076993","-14,783,562.087000","-9,427,199.012000","-6,165,270.744747","-32,314,522.524190","-4,800,000.000000","-156,329,474.444929"
2020,"-93,281,762.465546","-15,683,881.018098","-10,001,315.431831","-6,473,596.489566","-33,930,574.703931","-4,800,000.000000","-164,171,130.108972"
2021,"-97,946,791.801807","-16,639,029.372100","-10,610,395.541629","-6,797,341.632633","-35,627,445.798627","-4,800,000.000000","-172,421,004.146796"
2022,"-102,845,119.675026","-17,652,346.260861","-11,256,568.630115","-7,137,277.299441","-37,409,177.569486","-4,800,000.000000","-181,100,489.434930"
2023,"-107,988,413.366035","-18,727,374.148148","-11,942,093.659688","-7,494,213.179541","-39,280,013.906562","-4,800,000.000000","-190,232,108.259975"
2024,"-113,388,923.637428","-19,867,871.233770","-12,669,367.163564","-7,868,999.455130","-41,244,410.937230","-4,800,000.000000","-199,839,572.427121"
2025,"-119,059,513.913538","-21,077,824.591907","-13,440,931.623825","-8,262,528.826090","-43,307,047.640198","-4,800,000.000000","-209,947,846.595558"
2026,"-125,013,690.919711","-22,361,464.109554","-14,259,484.359715","-8,675,738.636311","-45,472,836.990319","-4,800,000.000000","-220,583,215.015610"


In [7]:
ops = [i for i in list(range(general.recent_end_year, general.recent_end_year+10))]
a_slider = widgets.SelectMultiple(options=ops, description='Years for comparison')
display(a_slider)

compare1 = ['NNB comparison','AUA comparison']
box_layout2 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='30%')
items3 = {}
for w3 in compare1:
    items3[w3] = widgets.Button(description=w3,layout=widgets.Layout(width='100%', height='40px'))
    items3[w3].style.button_color = 'lightgreen'
box3 = widgets.Box([list(items3.values())[0],list(items3.values())[1]],layout=box_layout2)
items3['NNB comparison'].on_click(lambda x: display(consolidated.get_nnb_compare(data_dic, input_dic,list(a_slider.value))))
items3['AUA comparison'].on_click(lambda x: display(consolidated.get_aua_compare(data_dic, input_dic,list(a_slider.value))))


display(box3)

SelectMultiple(description='Years for comparison', options=(2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027), value=())

Box(children=(Button(description='NNB comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='AUA comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='30%'))

In [8]:
compare2 = ['Interim revenue comparison','Annual revenue comparison', 'Interim costs comparison', 'Annual costs comparison']
box_layout3 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='50%')
items4 = {}
for w4 in compare2:
    items4[w4] = widgets.Button(description=w4,layout=widgets.Layout(width='100%', height='40px'))
    items4[w4].style.button_color = 'lightgreen'
top_box4 = widgets.HBox([list(items4.values())[0],list(items4.values())[1]],layout=box_layout3)
bottom_box4 = widgets.HBox([list(items4.values())[2],list(items4.values())[3]],layout=box_layout3)
items4['Interim revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,True,list(a_slider.value))))
items4['Annual revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,False,list(a_slider.value))))
items4['Interim costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,True,list(a_slider.value))))
items4['Annual costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,False,list(a_slider.value))))

widgets.VBox([top_box4, bottom_box4])

VBox(children=(HBox(children=(Button(description='Interim revenue comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%')), HBox(children=(Button(description='Interim costs comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))))

In [9]:
'''=====current values=======
discounted cash flow period: 9 years
perpetuity growth rate: 0.035
cash flow discounted rate: 0.085
'''
dcf_period = widgets.IntText(value=9,placeholder='Integer',description='DCF period - No. of years:')
display(dcf_period)
perp_growth = widgets.FloatText(value=0.035,placeholder='Percentage as decimals',description='Perpetuity growth rate:')
display(perp_growth)
disct_rate = widgets.FloatText(value=0.085,placeholder='Percentage as decimals',description='Cash flow discount rate:')
display(disct_rate)
current_value = widgets.Checkbox(value=False, description='Fair value now')
display(current_value)

dis_cash = ['Discounted cash flow','Fair value']
items5 = {}
for w5 in dis_cash:
    items5[w5] = widgets.Button(description=w5, layout=widgets.Layout(width='100%', height='40px'))
    items5[w5].style.button_color = 'lightgreen'
fair = widgets.HBox([list(items5.values())[0],list(items5.values())[1]],layout=box_layout3)
items5['Discounted cash flow'].on_click(lambda x: display(discf.disc_cash_flow(data_dic, input_dic,now=current_value.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value)))
items5['Fair value'].on_click(lambda x: display(discf.fair_value(data_dic, input_dic,now=current_value.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value,pep_rate=perp_growth.value)))    
display(fair)

IntText(value=9, description='DCF period - No. of years:')

FloatText(value=0.035, description='Perpetuity growth rate:')

FloatText(value=0.085, description='Cash flow discount rate:')

Checkbox(value=False, description='Fair value now')

HBox(children=(Button(description='Discounted cash flow', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Fair value', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))

In [10]:
radio1 = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(radio1)

others = ['HLF composite monthly return','Total NNB hlf Algo','NNB distribution hlf Algo','Total AUA','% Revenue distribution','Avg AUA distribution', 'Total NNB nnc Algo', 'NNB distribution nnc Algo']
items6 = {}
for w6 in others:
    items6[w6] = widgets.Button(description=w6, layout=widgets.Layout(width='100%', height='40px'))
    items6[w6].style.button_color = 'lightgreen'
other_box1 = widgets.VBox([list(items6.values())[0],list(items6.values())[1],list(items6.values())[2],list(items6.values())[3]],layout=box_layout)
other_box2 = widgets.VBox([list(items6.values())[4],list(items6.values())[5],list(items6.values())[6],list(items6.values())[7]],layout=box_layout)
items6['HLF composite monthly return'].on_click(lambda x: display(discretionary_aua.get_composite_return(data_dic)))
items6['Total NNB hlf Algo'].on_click(lambda x: display(combined.total_nnb(data_dic, input_dic, opt=radio1.value)))
items6['NNB distribution hlf Algo'].on_click(lambda x: display(combined.nnb_distribution(data_dic, input_dic, opt=radio1.value)))
items6['Total AUA'].on_click(lambda x: display(combined.total_aua(data_dic, input_dic)))
items6['% Revenue distribution'].on_click(lambda x: display(stats.summary_revenue_dist_percent(data_dic, input_dic)))
items6['Avg AUA distribution'].on_click(lambda x: display(stats.summary_avg_aua_dist(data_dic, input_dic,radio1.value)))
items6['Total NNB nnc Algo'].on_click(lambda x: display(combined.total_nnb_ci_clientAlgo(data_dic, input_dic, opt=radio1.value)))
items6['NNB distribution nnc Algo'].on_click(lambda x: display(combined.total_nnb_distribution_clientAlgo(data_dic, input_dic, opt=radio1.value)))

widgets.HBox([other_box1, other_box2])

#Financial year based

RadioButtons(description='Period', options=('month_no', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='month_no')

HBox(children=(VBox(children=(Button(description='HLF composite monthly return', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total NNB hlf Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='NNB distribution hlf Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total AUA', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='% Revenue distribution', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Avg AUA distribution', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total NNB nnc Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='NNB distribution nnc Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

In [11]:
nnb_radio = widgets.RadioButtons(options=['day','month','semi-annual','quarter','annual'],description='Period',disabled=False)
nnb_dropdown = widgets.Dropdown(options=['no_select',None],description='Fund option',disabled=False)
unit_dropdown = widgets.Dropdown(options=['acc','inc'],description='Unit type',disabled=False)
display(nnb_radio)
display(nnb_dropdown)
display(unit_dropdown)

nnb_b = widgets.Button(description='Implied HLF nnb period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b.style.button_color = 'lightgreen'

nnb_b2 = widgets.Button(description='HLF unit change period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b2.style.button_color = 'lightgreen'
nnb_b.on_click(lambda x: display(stats.hlf_to_date_implied_nnb(data_dic,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))
nnb_b2.on_click(lambda x: display(stats.hlf_to_date_unit_change(data_dic, unit_type=unit_dropdown.value,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))

display(nnb_b)
display(nnb_b2)

RadioButtons(description='Period', options=('day', 'month', 'semi-annual', 'quarter', 'annual'), value='day')

Dropdown(description='Fund option', options=('no_select', None), value='no_select')

Dropdown(description='Unit type', options=('acc', 'inc'), value='acc')

Button(description='Implied HLF nnb period-to-date', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

Button(description='HLF unit change period-to-date', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

In [12]:
cash_radio = widgets.RadioButtons(options=['monthly','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(cash_radio)


cash_b = widgets.Button(description='Cash margin', layout=widgets.Layout(width='50%', height='40px'))
cash_b.style.button_color = 'lightgreen'
cash_b.on_click(lambda x: display(stats.cash_margin(data_dic,cash_radio.value)))
display(cash_b)

RadioButtons(description='Period', options=('monthly', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='monthly')

Button(description='Cash margin', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

In [13]:
eps_radio = widgets.RadioButtons(options=[True, False],description='Calendar year?',disabled=False)
display(eps_radio)

eps_summary = widgets.Button(description='Net earnings and EPS', layout=widgets.Layout(width='50%', height='40px'))
eps_summary.style.button_color = 'lightgreen'
eps_summary.on_click(lambda x: display(stats.summary_total(data_dic, input_dic, cal=eps_radio.value)))
display(eps_summary)


RadioButtons(description='Calendar year?', options=(True, False), value=True)

Button(description='Net earnings and EPS', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

,Total Revenue,Total Costs,Net Earning Before Tax,Net Earning After Tax,EPS
calendar_year,,,,,
2018,"493,810,563.471675","-154,955,337.953000","338,855,225.518675","274,472,732.670127",0.578565
2019,"582,975,880.610448","-160,200,569.497544","422,775,311.112905","342,448,002.001453",0.721851
2020,"625,345,294.415444","-168,243,728.563849","457,101,565.851595","370,252,268.339792",0.780460
2021,"694,720,644.296221","-176,705,664.901264","518,014,979.394957","419,592,133.309915",0.884464
2022,"770,901,661.802664","-185,608,328.772895","585,293,333.029770","474,087,599.754113",0.999335
2023,"853,305,670.700761","-194,974,829.513378","658,330,841.187383","533,247,981.361780",1.124040
2024,"933,644,766.602388","-204,829,497.229577","728,815,269.372811","590,340,368.191977",1.244386
2025,"1,007,235,123.577247","-215,197,947.814412","792,037,175.762834","641,550,112.367896",1.352332
2026,"1,086,119,674.052259","-226,107,151.453382","860,012,522.598877","696,610,143.305090",1.468394


,Total Revenue,Total Costs,Net Earning Before Tax,Net Earning After Tax,EPS
financial_year,,,,,
2018,"452,875,058.411950","-148,646,100.000000","304,228,958.411950","246,425,456.313680",0.519443
2019,"542,126,139.381681","-156,329,474.444929","385,796,664.936752","312,495,298.598769",0.658713
2020,"604,118,100.072234","-164,171,130.108972","439,946,969.963262","356,357,045.670242",0.751170
2021,"659,121,304.607056","-172,421,004.146796","486,700,300.460261","394,227,243.372811",0.830997
2022,"731,932,795.778705","-181,100,489.434930","550,832,306.343775","446,174,168.138458",0.940496
2023,"811,585,219.784860","-190,232,108.259975","621,353,111.524886","503,296,020.335157",1.060904
2024,"894,918,144.183042","-199,839,572.427121","695,078,571.755920","563,013,643.122296",1.186784
2025,"969,790,469.624293","-209,947,846.595558","759,842,623.028735","615,472,524.653275",1.297363
2026,"1,045,950,617.878697","-220,583,215.015610","825,367,402.863088","668,547,596.319101",1.409240


In [14]:
hlf_radio = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(hlf_radio)


buttom3 = ['HLF revenue margin','Monthly revenue', 'HLF avg fund size','Quarter Trading update revenue']
items7 = {}
for w7 in buttom3:
    items7[w7] = widgets.Button(description=w7, layout=widgets.Layout(width='100%', height='40px'))
    items7[w7].style.button_color = 'lightgreen'

top_m = widgets.HBox([list(items7.values())[0],list(items7.values())[1]],layout=box_layout3)
bottom_m = widgets.HBox([list(items7.values())[2],list(items7.values())[3]],layout=box_layout3)

items7['HLF revenue margin'].on_click(lambda x: display(stats.hlf_revenue_margin(data_dic,input_dic,hlf_radio.value)))
items7['Monthly revenue'].on_click(lambda x: display(revenue.monthly_revenue(data_dic, input_dic)))
items7['HLF avg fund size'].on_click(lambda x: display(stats.avg_hlf_size(data_dic, input_dic,hlf_radio.value)))
items7['Quarter Trading update revenue'].on_click(lambda x: display(stats.quarter_revenue(data_dic, input_dic)))


widgets.VBox([top_m, bottom_m])

RadioButtons(description='Period', options=('month_no', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='month_no')

VBox(children=(HBox(children=(Button(description='HLF revenue margin', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Monthly revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%')), HBox(children=(Button(description='HLF avg fund size', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Quarter Trading update revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))))

In [15]:
display(a_slider)

client_options = ['Total No. of clients','Net new clients']
items8 = {}
for w8 in client_options:
    items8[w8] = widgets.Button(description=w8, layout=widgets.Layout(width='100%', height='40px'))
    items8[w8].style.button_color = 'lightgreen'
c_fair = widgets.HBox([list(items8.values())[0],list(items8.values())[1]],layout=box_layout3)
items8['Total No. of clients'].on_click(lambda x: display(combined.total_client_predt(data_dic, input_dic).loc[list(a_slider.value)]))
items8['Net new clients'].on_click(lambda x: display(combined.net_new_client_predt(data_dic, input_dic).loc[list(a_slider.value)]))
display(c_fair)

SelectMultiple(description='Years for comparison', options=(2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027), value=())

HBox(children=(Button(description='Total No. of clients', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Net new clients', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))

In [16]:
# %load_ext autoreload
# %autoreload 2

In [17]:
vantage_aua.hl_shares_aua(data_dic)

2017-12-29   3,400,000,000.000000
2018-01-31   3,504,716,981.132075
2018-02-28   3,255,660,377.358490
2018-03-30   3,081,132,075.471698
2018-04-30   3,376,415,094.339623
2018-05-31   3,598,113,207.547170
2018-06-29   3,719,811,320.754717
2018-07-31                    nan
2018-08-31                    nan
2018-09-28                    nan
2018-10-31                    nan
2018-11-30                    nan
2018-12-31                    nan
2019-01-31                    nan
2019-02-28                    nan
2019-03-29                    nan
2019-04-30                    nan
2019-05-31                    nan
2019-06-28                    nan
2019-07-31                    nan
2019-08-30                    nan
2019-09-30                    nan
2019-10-31                    nan
2019-11-29                    nan
2019-12-31                    nan
2020-01-31                    nan
2020-02-28                    nan
2020-03-31                    nan
2020-04-30                    nan
2020-05-29    

In [18]:
'''
fair value sensitivity analysis

disc_rate_list = [0.0825,0.085,0.0875,0.09]
pep_rate_list = [0.035,0.0375,0.04,0.0425]

df_now = pandas.DataFrame(columns = disc_rate_list, index=pep_rate_list)
df_now.columns.name = 'Cost of Equity'
df_now.index.name = 'Long-term Growth Rate'

for i in pep_rate_list:
    for j in disc_rate_list:
        df_now.loc[i,j] = discf.fair_value(data_dic, input_dic,now=True,dcf_p=9,disc_rate=j,pep_rate=i).loc['Fair value per share','HL']
df_1y = pandas.DataFrame(columns = disc_rate_list, index=pep_rate_list)
df_1y.columns.name = 'Cost of Equity'
df_1y.index.name = 'Long-term Growth Rate'

for i in pep_rate_list:
    for j in disc_rate_list:
        df_1y.loc[i,j] = discf.fair_value(data_dic, input_dic,now=False,dcf_p=9,disc_rate=j,pep_rate=i).loc['Fair value per share','HL']
'''

In [26]:
#consolidated.annual_revenue_analysis(data_dic, input_dic)
revenue.annual_revenue(data_dic, input_dic)

,renewal_income,management_fee,stockbroking_commission,stockbroking_income,interest_on_cash,hlf_amc,platform_fee,pms_advice,advice_fee,funds_library,paper_income,other_income,currency_revenue,interest_on_reserve,cash_service
financial_year,,,,,,,,,,,,,,,
2018,"5,394,600.000000","19,744,060.000000","71,098,800.000000","2,626,000.000000","43,374,225.536675","67,325,161.479452","198,336,046.992576","13,098,998.803247","11,400,000.000000","8,040,000.000000","5,029,985.600000","3,784,000.000000","2,211,000.000000","1,412,180.000000",0.000000
2019,"5,373,043.178400","20,328,801.857925","78,836,493.767892","2,732,090.400000","78,841,282.964364","75,385,966.940183","232,589,713.872120","15,051,308.885179","11,400,000.000000","8,201,604.000000","5,781,516.000000","3,784,000.000000","2,255,441.100000","1,461,751.415617","103,125.000000"
2020,"5,351,572.497859","20,930,861.483342","82,779,113.916509","2,842,466.852160","107,719,532.634527","85,195,581.924003","248,782,852.057326","16,149,600.075260","11,400,000.000000","8,366,456.240400","6,614,724.000000","3,784,000.000000","2,300,775.466110","1,513,062.924739","387,500.000000"
2021,"5,330,187.614158","21,550,751.760810","86,918,904.853594","2,957,302.512987","124,669,670.534757","94,792,339.850619","269,726,668.692185","17,288,148.914685","11,400,000.000000","8,534,622.010832","7,568,011.200000","3,784,000.000000","2,347,021.052979","1,566,175.609451","687,500.000000"
2022,"5,308,888.184451","22,189,000.764526","91,265,727.108023","3,076,777.534512","142,864,427.809736","105,206,606.042845","305,961,323.983068","18,508,345.966162","11,400,000.000000","8,706,167.913250","8,658,681.600000","3,784,000.000000","2,394,196.176144","1,621,152.695987","987,500.000000"
2023,"5,292,966.834101","22,846,152.208178","95,829,934.334611","3,201,079.346906","163,006,356.774435","116,252,088.407947","345,970,676.839903","19,816,351.201191","11,400,000.000000","8,881,161.888306","9,896,572.800000","3,784,000.000000","2,442,319.519284","1,678,059.629998","1,287,500.000000"
2024,"5,292,966.834101","23,522,765.908129","100,622,397.975379","3,330,402.952521","183,930,152.070794","128,329,783.173384","387,582,328.099179","21,195,557.231216","11,400,000.000000","9,059,673.242261","11,052,242.400000","3,784,000.000000","2,491,410.141622","1,736,964.154455","1,587,500.000000"
2025,"5,292,966.834101","24,219,418.260313","105,654,533.154143","3,464,951.231803","201,928,247.333683","140,379,355.844399","423,867,771.458282","22,586,176.957373","11,400,000.000000","9,241,772.674430","11,744,352.000000","3,784,000.000000","2,541,487.485468","1,797,936.390298","1,887,500.000000"
2026,"5,292,966.834101","24,936,702.731258","110,938,325.866090","3,604,935.261568","219,836,315.873807","153,528,024.252847","460,141,430.502803","24,040,272.747209","11,400,000.000000","9,427,532.305187","12,378,991.200000","3,784,000.000000","2,592,571.383926","1,861,048.919902","2,187,500.000000"
